# Agenda

1. Detection

    * Different tasks/taxonomy
    * Conceptual model
    * Interface

2. `skchange` algorithm framework

    * Interval scoring
    * Search

3. Interval scoring

    * Costs
    * Change scores
    * Anomaly scores

4. Change detection

5. Segment anomaly detection

# The detection module
Experimental module, still under heavy development.

Contributions appreciated!

# Detection tasks

1. Change detection
2. Segmentation
3. Point anomaly/outlier detection
4. Segment anomaly detection


## Change detection and segmentation

![Change detection and segmentation](img/changepoints_and_segments.png)

* Change detection: Detect points in time where the data generating process changes significantly.
* Segmentation: Divide the data into segments based on certain criteria. The same label can be applied at multiple disconnected segments.

### Use cases

* Data cleaning: Remove segments that are not relevant for the analysis.
* Preprocessing: Divide the data into homogenous parts for individual analysis.
* Detect interesting patterns: Anomaly detection, motif discovery, state transitions.

## Point and segment anomaly detection

![Point and Segment Anomalies](img/point_vs_segment_anomaly.png)

* Point anomaly detection: Detect individual data points that are significantly different from the rest of the data.
* Segment anomaly detection: Detect segments of data that are significantly different from the rest of the data.

### Use cases

* Data cleaning: Remove anomalies from the data.
* Detect interesting events: Fault detection, fraud detection, etc.

# Detector conceptual model

1. Set hyperparameters
2. Fit the detector to training data
3. Detect events on new data

    * Input: A time series
    * Output: Detected events.

        - Change points
        - Segments
        - Point anomalies
        - Segment anomalies
        - Change intervals
        - ... 

# Detector interface

* `__init__(self, ...)`

    - Set hyperparameters.
* `fit(self, X, y=None)`

    - Fit the detector to the training data.
* `predict(self, X)`

    - Detect events on new data. 
    - Sparse format: One entry per detected event.
* `transform(X)` 

    - Detect events on new data. 
    - Dense format: One entry per input time point.
    - Default: Run `predict` + sparse to dense conversion.
* `transform_scores(X)` [optional] 

    - Return detection scores for each time point.